# Introduction to Prompt Engineering

In this tutorial, you'll learn two basic techniques to write effective prompts for large language models (LLMs).


In [2]:
import openai
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") 
openai.api_key  = os.environ["OPENAI_API_KEY"]

### ChatCompletion API ###
First, we will introduce how to use the OpenAI ChatCompletion API (Completion is going to depreciate).  
Let's take a look at their documentation:  
Guide: https://platform.openai.com/docs/guides/gpt/chat-completions-api  
API: https://platform.openai.com/docs/api-reference/chat/object




In [3]:
# def get_completion(prompt, model="gpt-4"):
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

Let's try to get the agent to summarise a text for us. We will use **Prompts** do to so. A prompt is essentially the text we feed into the LLM. It can include context, task, output structure and other specific instructions.

Be specific and clearly break down different parts of the prompt


In [4]:
text = f"""
This coursework assignment explores the design and development of your software
project. The deliverable here is a proposal which defines and describes a number of
key elements of your project. You should produce a document that is no longer than
8,000 words in length (not including appendix) and should be submitted as a single
PDF document.
This proposal should be used to explore design decisions, consider the context of use
and identify the process by which the software project is developed. As such, your
proposal should include a reasoned justification that explores the following topics:
1) A clearly defined set of deliverable components of the software and the job of
work required to bring these components to completion through research and
iterative prototyping informed by said research.
2) The defined timescale of work, including any dependencies, milestones or
contingencies. There should be a clear narrative describing your breakdown of
work and activities as part of the project.
3) A formal specification of the desired system. This will include a description of
how your iterative design exercises have converged on an appropriate,
relevant and acceptable solution. You should also include user-acceptance
criteria for testing at this stage.
4) A clearly defined scope for the project defining areas that you will and will not
be delivering on.
5) Some evidence of requirements elicitation involving some/all of your project
stakeholders. Evidence should be provided supporting your decision making
at all times. This will come either in the form of existing empirical evidence
from literature sources or through empirical proof through techniques such as
usability studies and heuristic evaluations that you have performed.
6) A research summary that highlights the challenges of working within your
chosen domain and summarises the capabilities and competencies of similar
tools.
7) A description of your approach that discusses the motivations and reasoning
for working in a particular manner (e.g. User-Centred Design, Test-Driven
Development.) This should also include a clear and descriptive narrative that
identifies the tasks required to bring this project to fruition e.g. as a backlog,
set of use cases or functional/technical specifications supported by evidenced
based reasoning.
8) Some early prototypes showing iterative design and development activities
that identify how the project will work and highlighting the strengths and
weaknesses of your proposition.
9) Some early evidence of assumption testing and validation of your designs to
date (e.g. user tests or automated feedback such as W3C
validation/accessibility testing, heuristic tests etc.)
10)A critical evaluation of your concept, your project in its current state and the
proposed software project.
The document should highlight a clear and systematic rhetoric with critical analysis
and an overall evaluation regarding the current state and feasibility of the approach
presented.
"""

prompt = f"""
List down the actionable items from the project proposal requirements delimited by triple backticks (```).

```{text}```
"""

response = get_completion(prompt)
print(response)

1) Define and describe the deliverable components of the software and the necessary research and iterative prototyping required to complete them.
2) Create a defined timescale of work, including dependencies, milestones, and contingencies. Provide a clear narrative describing the breakdown of work and activities.
3) Develop a formal specification of the desired system, including a description of how iterative design exercises have led to an appropriate solution. Include user-acceptance criteria for testing.
4) Clearly define the scope of the project, specifying what will and will not be delivered.
5) Elicit requirements from project stakeholders and provide evidence supporting decision making. Use empirical evidence from literature sources or usability studies and heuristic evaluations.
6) Summarize the challenges of working within the chosen domain and describe the capabilities and competencies of similar tools through research.
7) Describe the approach for the project, discussing mot

Sometimes we may want to use the output directly in a program. We can add the structure requirement to the prompt.

In [35]:
prompt = f"""
The project proposal requirements are privided below, delimited by triple backticks (```).  
List the actionable items in JSON format.

```{text}```
"""
response = get_completion(prompt)
print(response)


{
  "actionable_items": [
    "Define a set of deliverable components of the software",
    "Research and iterate on prototypes to bring the components to completion",
    "Define the timescale of work, including dependencies, milestones, and contingencies",
    "Breakdown the work and activities as part of the project",
    "Create a formal specification of the desired system",
    "Converge on an appropriate, relevant, and acceptable solution through iterative design exercises",
    "Include user-acceptance criteria for testing",
    "Define the scope of the project, including areas to be delivered on and areas not to be delivered on",
    "Elicit requirements from project stakeholders",
    "Provide evidence supporting decision making",
    "Summarize the challenges of working within the chosen domain",
    "Summarize the capabilities and competencies of similar tools",
    "Describe the approach for working in a particular manner (e.g. User-Centred Design, Test-Driven Development)"

### Few-shot prompting ###
We can also provide some examples in the prompt for the LLM to learn. Formally it's call "Few-shot" prompting.

In [36]:
examples = """{
    "tasks": {
        "1": {
            "description": "Define deliverable components and required work.",
            "est time": "2 days",
            "dependency": "NA",
        },
        "2": {
            "description": "Specify timescale, dependencies, milestones, and contingencies",
            "est time": "3 days",
            "depencency": "1"
        }
    }
}
"""
prompt = f"""
The project proposal requirements are privided below, delimited by triple backticks (```). 
Create the actionable items in JSON format, using the following examples delimited by tag <examples> and </examples>.

<examples>
{examples}
</examples>


```{text}```
"""
response = get_completion(prompt)
print(response)

{
    "tasks": {
        "1": {
            "description": "Define deliverable components and required work.",
            "est time": "2 days",
            "dependency": "NA"
        },
        "2": {
            "description": "Specify timescale, dependencies, milestones, and contingencies",
            "est time": "3 days",
            "dependency": "1"
        },
        "3": {
            "description": "Create a formal specification of the desired system",
            "est time": "4 days",
            "dependency": "2"
        },
        "4": {
            "description": "Define the scope of the project",
            "est time": "1 day",
            "dependency": "3"
        },
        "5": {
            "description": "Elicit requirements from project stakeholders",
            "est time": "2 days",
            "dependency": "4"
        },
        "6": {
            "description": "Conduct research on challenges within the chosen domain and capabilities of similar tools",
      

### Chain of Thoughts(CoT) ###

You can specify the steps that the LLM should take to generate the final output.   
This is a technique called **Chain of Thoughts (CoT)**

In [37]:
prompt = f"""
Perform the following actions:
1. Summarize a list of actionable items from the project proposal requirements, delimited by triple backticks (```).
2. For each of the item, include the IDs of its dependent items.
3. Estimate the time required to complete the item, assuming the dependencies are completed.
4. Create the actionable items in JSON format, using the following examples delimited by tag <examples> and </examples>.

<examples>
{examples}
</examples>


```{text}```
"""

response = get_completion(prompt)
print(response)

```
{
    "tasks": {
        "1": {
            "description": "Define deliverable components and required work.",
            "est time": "2 days",
            "dependency": "NA"
        },
        "2": {
            "description": "Specify timescale, dependencies, milestones, and contingencies",
            "est time": "3 days",
            "dependency": "1"
        },
        "3": {
            "description": "Create a formal specification of the desired system",
            "est time": "4 days",
            "dependency": "2"
        },
        "4": {
            "description": "Define the scope of the project",
            "est time": "1 day",
            "dependency": "2"
        },
        "5": {
            "description": "Gather evidence of requirements elicitation",
            "est time": "2 days",
            "dependency": "2"
        },
        "6": {
            "description": "Summarize research challenges and capabilities of similar tools",
            "est time": "1 day

## Use LLM for NLP tasks ##
Traditionally, for each NLP task, such as classification, summarization, and translation, you need to train a dedicated model to perform the task. 

With the help of LLM, you can now use prompt do to all these tasks. 

### Classification ###

In [7]:
examples = """{
    "tasks": {
        "1": {
            "description": "Define deliverable components and required work.",
            "est time": "2 days",
            "dependency": "NA",
            "category": "scoping",
            "difficulty": "easy"
        },
        "2": {
            "description": "Specify timescale, dependencies, milestones, and contingencies",
            "est time": "3 days",
            "depencency": "1",
            "category": "scoping",
            "difficulty": "easy"
        }
    }
}
"""

prompt = f"""
Perform the following actions:
1. Summarize a list of actionable items from the project proposal requirements, delimited by triple backticks (```).
2. For each of the item, include the IDs of its dependent items.
3. Estimate the time required to complete the item, assuming the dependencies are completed.
4. Classify the tasks into different categories, such as "design", "development", "testing", "documentation", "deployment", etc.
5. Classify the tasks into different difficulties, such as "easy", "medium", "hard", "very hard", etc.
6. Create the actionable items in JSON format, using the following examples delimited by tag <examples> and </examples>.

<examples>
{examples}
</examples>


```{text}```
"""

response = get_completion(prompt)
print(response)

```
{
    "tasks": {
        "1": {
            "description": "Define deliverable components and required work.",
            "est time": "2 days",
            "dependency": "NA",
            "category": "scoping",
            "difficulty": "easy"
        },
        "2": {
            "description": "Specify timescale, dependencies, milestones, and contingencies",
            "est time": "3 days",
            "dependency": "1",
            "category": "scoping",
            "difficulty": "easy"
        },
        "3": {
            "description": "Formal specification of the desired system",
            "est time": "4 days",
            "dependency": "2",
            "category": "design",
            "difficulty": "medium"
        },
        "4": {
            "description": "Define project scope",
            "est time": "1 day",
            "dependency": "2",
            "category": "scoping",
            "difficulty": "easy"
        },
        "5": {
            "description": "Evi

### Summarisation ###

In [6]:
prompt = f"""
Summarize the given project proposal requirements, delimited by triple backticks (```), into a paragraph of 200 words.
```{text}```
"""
response = get_completion(prompt)
print(response)

The project proposal requirements for this coursework assignment involve designing and developing a software project. The proposal should be no longer than 8,000 words and should be submitted as a single PDF document. The proposal should explore design decisions, consider the context of use, and identify the software development process. It should include a justification for the deliverable components of the software and the necessary research and iterative prototyping to complete them. The proposal should also define the timescale of work, including dependencies, milestones, and contingencies, with a clear breakdown of work and activities. A formal specification of the desired system should be included, along with user-acceptance criteria for testing. The scope of the project should be clearly defined, indicating what will and will not be delivered. Evidence of requirements elicitation involving project stakeholders should be provided, either through literature sources or empirical pr

### translation ###

In [8]:
summary = """
The project proposal requirements for this coursework assignment involve designing and developing a software project. The proposal should be no longer than 8,000 words and should be submitted as a single PDF document. The proposal should explore design decisions, consider the context of use, and identify the software development process. It should include a justification for the deliverable components of the software and the necessary research and iterative prototyping to complete them. The proposal should also define the timescale of work, including dependencies, milestones, and contingencies, with a clear breakdown of work and activities. A formal specification of the desired system should be included, along with user-acceptance criteria for testing. The scope of the project should be clearly defined, indicating what will and will not be delivered. Evidence of requirements elicitation involving project stakeholders should be provided, either through literature sources or empirical proof such as usability studies. A research summary should highlight the challenges of the chosen domain and the capabilities of similar tools. The approach to the project should be described, including motivations and reasoning, and the tasks required to complete the project should be identified. Early prototypes should demonstrate iterative design and development activities, highlighting strengths and weaknesses. Assumption testing and validation of designs should also be included. Finally, a critical evaluation of the concept, current state of the project, and proposed software project should be provided, with a clear and systematic rhetoric and overall evaluation of feasibility.
"""

prompt = f"""
Translate the given summary, delimited by triple backticks (```), into simplified chinese.
```{summary}```
"""
response = get_completion(prompt)
print(response)

这个课程作业的项目提案要求设计和开发一个软件项目。提案的字数不应超过8000字，并且应以单个PDF文档的形式提交。提案应探讨设计决策，考虑使用环境，并确定软件开发过程。它应包括对软件交付组件的合理化解释以及完成它们所需的研究和迭代原型制作。提案还应定义工作的时间表，包括依赖关系、里程碑和应急措施，并清楚地分解工作和活动。应包括所需系统的正式规范，以及用于测试的用户验收标准。项目的范围应明确定义，指明将要交付和不会交付的内容。应提供与项目利益相关者进行需求获取的证据，可以是通过文献来源或可用性研究等经验证明。研究摘要应突出所选择领域的挑战和类似工具的能力。应描述项目的方法，包括动机和推理，并确定完成项目所需的任务。早期原型应展示迭代设计和开发活动，突出优点和缺点。还应包括假设测试和设计验证。最后，应提供对概念、项目当前状态和拟议软件项目的批判性评估，以及对可行性的明确和系统的论证和总体评估。


## Chat ##
We've been using the **ChatCompletion** API to do various one-step NLP tasks. Now let's learn how to build a chat that have extended converstaions with chatbots.

In [9]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

messages =  [  
{'role':'system', 'content':'You are a university computer science professor that aims to help students pass their assessments.'},    
{'role':'user', 'content':"""
Can you guide me through the process of writing a project proposal? A summary of the project requirement is as follows:

The project proposal requirements for this coursework assignment involve designing and developing a software project. The proposal should be no longer than 8,000 words and should be submitted as a single PDF document. The proposal should explore design decisions, consider the context of use, and identify the software development process. It should include a justification for the deliverable components of the software and the necessary research and iterative prototyping to complete them. The proposal should also define the timescale of work, including dependencies, milestones, and contingencies, with a clear breakdown of work and activities. A formal specification of the desired system should be included, along with user-acceptance criteria for testing. The scope of the project should be clearly defined, indicating what will and will not be delivered. Evidence of requirements elicitation involving project stakeholders should be provided, either through literature sources or empirical proof such as usability studies. A research summary should highlight the challenges of the chosen domain and the capabilities of similar tools. The approach to the project should be described, including motivations and reasoning, and the tasks required to complete the project should be identified. Early prototypes should demonstrate iterative design and development activities, highlighting strengths and weaknesses. Assumption testing and validation of designs should also be included. Finally, a critical evaluation of the concept, current state of the project, and proposed software project should be provided, with a clear and systematic rhetoric and overall evaluation of feasibility.
""" 
 },   
]

response = get_completion_from_messages(messages, temperature=1)
print(response)

Sure! Here's a step-by-step guide to help you write a project proposal for your software development coursework assignment:

1. Introduction:
   - Provide an overview of the project, including its purpose and objectives.
   - Explain the relevance and importance of the project in the chosen domain.
   - Introduce the problem or challenge that the software aims to address.

2. Context of Use:
   - Identify and describe the target users or stakeholders of the software.
   - Analyze their needs, preferences, and constraints to guide design decisions.
   - Consider any environmental or situational factors that may impact software usage.

3. Software Development Process:
   - Select an appropriate software development methodology/framework.
   - Justify the chosen methodology and explain how it complements the project requirements.
   - Outline the key stages, activities, and deliverables of the software development process.

4. Deliverable Components:
   - Specify the different functionali

In [10]:
messages.append({'role':'assistant', 'content':response})
messages.append({'role':'user', 'content':'Can you generate a proposal template?'})

response = get_completion_from_messages(messages, temperature=1)
print(response)

Certainly! Here's a template for a project proposal that you can use as a starting point:

[Your Name]
[Your Affiliation]
[Date]

[Title of the Proposal]

1. Introduction
   - Background information on the project and its relevance
   - Objectives and goals of the project
   - Problem statement and its significance in the chosen domain

2. Context of Use
   - Target users/stakeholders and their needs
   - Factors influencing software usage (environmental, situational, etc.)

3. Software Development Process
   - Selected methodology/framework and justification
   - Stages, activities, and deliverables of the software development process

4. Deliverable Components
   - Functionality and features of the software
   - Justification for the inclusion of each component
   - Research and iterative prototyping required for development

5. Timescale and Work Breakdown
   - Estimated project timeline
   - Dependencies, milestones, and contingencies
   - Breakdown of work and activities

6. Forma

### Model Limitation: Hallucinations ###
If the task is too complex and the instruction is not clear, the LLM may confidently respond with unreal information.  
In this example, **LLama 2** was not released when GPT-3.5 or GPT-4 was released.   

Let's see what LLM thinks LLama 2 is.

In [11]:
prompt = f"""
What is so special about Llama 2?
"""
response = get_completion(prompt)
print(response)

Llama 2 is a fictional character created by the OpenAI team to showcase the capabilities of their language model, GPT-3. It is not a real llama but a representation of the AI's ability to generate creative and engaging content. Llama 2 is special because it demonstrates the potential of AI in various applications such as writing, storytelling, and problem-solving. It can generate text in a conversational manner, answer questions, and provide information on a wide range of topics. The uniqueness of Llama 2 lies in its ability to understand and respond to human prompts, making it an impressive example of AI technology.


Here's what LLama 2 actually is: https://ai.meta.com/llama/

How do we tackle LLM's Hallucinations? Stay tuned!